In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

In [ ]:
def load_workbook(xlsx_path: Path):
    assert xlsx_path.exists(), f"File not found: {xlsx_path}"
    xls = pd.ExcelFile(xlsx_path)
    return xls, xls.sheet_names


def parse_schools(xlsx_path: Path) -> pd.DataFrame:
    """
    Analysis of school locations.
    """
    raw = pd.read_excel(xlsx_path, sheet_name="School Locations", header=None)
    tmp = raw.iloc[9:].reset_index(drop=True)
    tmp.columns = ["School", "X", "Y"]
    mask = tmp["School"].astype(str).str.match(r"School\s+\d+")
    df = (
        tmp.loc[mask, ["School", "X", "Y"]]
        .assign(school_id=lambda d: d["School"].str.extract(r"(\d+)").astype(int))
        .drop(columns=["School"])
        .astype({"X": int, "Y": int})
        .sort_values("school_id")
        .reset_index(drop=True)
    )
    return df


def parse_patients_and_severity(xlsx_path: Path) -> pd.DataFrame:
    """
    Analysis of patients and their severity levels.
    """
    # 宽→长
    patients_wide = pd.read_excel(xlsx_path, sheet_name="Case_PatientList")
    pat_long = (
        patients_wide
        .melt(var_name="school_col", value_name="PATIENT")
        .dropna(subset=["PATIENT"])
        .assign(school=lambda d: d["school_col"].str.extract(r"#(\d+)").astype(int))
        .drop(columns=["school_col"])
    )

    severity_tbl = pd.read_excel(xlsx_path, sheet_name="Case_PatientSeverity")

    # 严重度映射（结合数据中出现的标签）
    sev_map = {
        "Intermittent": 1,
        "MildIntermittent": 1,
        "MildPersistent": 2,
        "ModeratePersistent": 3,
        "SeverePersistent": 4,
    }

    patients = (
        pat_long.merge(severity_tbl, on="PATIENT", how="left")
        .assign(
            pid=lambda d: d["PATIENT"].str.extract(r"(\d+)").astype(int),
            sev_weight=lambda d: d["SEVERITY"].map(sev_map).fillna(2).astype(int),
        )
        [["pid", "PATIENT", "school", "SEVERITY", "sev_weight"]]
        .sort_values(["school", "pid"])
        .reset_index(drop=True)
    )
    return patients


def build_school_summary(schools: pd.DataFrame, patients: pd.DataFrame) -> pd.DataFrame:
    """
    School-level summary: school_id, X, Y, num_patients, MildIntermittent, MildPersistent, ModeratePersistent, SeverePersistent
    其中各严重度列为该校患者数。
    """
    counts = patients.groupby("school")["pid"].count().rename("num_patients").reset_index()

    sev_dist = (
        patients.pivot_table(
            index="school", columns="SEVERITY", values="pid", aggfunc="count", fill_value=0
        )
        .reset_index()
    )

    df = (
        schools.merge(counts, left_on="school_id", right_on="school", how="left")
        .merge(sev_dist, left_on="school_id", right_on="school", how="left")
        .drop(columns=["school_x", "school_y"], errors="ignore")
        .rename(columns={"school_id": "school"})
        .fillna(0)
    )

    if "num_patients" in df.columns:
        df["num_patients"] = df["num_patients"].astype(int)

    cols = ["school", "X", "Y", "num_patients"]
    other_cols = [c for c in df.columns if c not in cols]
    df = df[cols + other_cols].sort_values("school").reset_index(drop=True)
    return df


def estimate_capacity_from_appointments(xlsx_path: Path, sheet_names) -> pd.DataFrame:
    """
    Get historical capacity estimates from appointment sheets.
    """
    app_sheets = [s for s in sheet_names if s.startswith("Case_SCH#")]
    apps = []
    for s in app_sheets:
        sid = int(s.split("#")[1].split("_")[0])
        df = pd.read_excel(xlsx_path, sheet_name=s)
        df["school"] = sid
        apps.append(df)
    if not apps:
        return pd.DataFrame(columns=["van", "daily_avg", "monthly_capacity_est"])

    apps = pd.concat(apps, ignore_index=True)
    apps["APP DATE"] = pd.to_datetime(apps["APP DATE"], errors="coerce")
    apps["VAN"] = apps["VAN"].astype(str)

    daily = apps.groupby(["VAN", "APP DATE"]).size().reset_index(name="appointments")
    cap_daily = daily.groupby("VAN")["appointments"].mean().round(2)
    cap_monthly = (cap_daily * 20).round(0).astype("Int64")

    cap_summary = (
        pd.DataFrame({"daily_avg": cap_daily, "monthly_capacity_est": cap_monthly})
        .reset_index()
        .rename(columns={"VAN": "van"})
        .sort_values("van")
        .reset_index(drop=True)
    )
    return cap_summary

def export_school_severity_matrix(
    patients: pd.DataFrame,
    out_dir: Path,
    filename: str = "Q1_school_severity_matrix.xlsx",
    heatmap_png: str = "Q1_heatmap_school_severity.png",
):
    """
    return: (matrix DataFrame, heatmap PNG path)
    """
 
    sev_order = ["MildIntermittent", "MildPersistent",
                 "ModeratePersistent", "SeverePersistent"]

    # 计数矩阵
    mat = (
        patients.assign(
            SEVERITY=pd.Categorical(patients["SEVERITY"], categories=sev_order, ordered=True)
        )
        .pivot_table(index="school", columns="SEVERITY", values="pid",
                     aggfunc="count", fill_value=0)
        .sort_index()
    )
 
    xlsx_path = out_dir / filename
    with pd.ExcelWriter(xlsx_path) as writer:
        mat.to_excel(writer, sheet_name="counts")
        (mat.sum(axis=1).rename("Total_by_school")
         .to_frame().to_excel(writer, sheet_name="totals_by_school"))
        (mat.sum(axis=0).rename("Total_by_severity")
         .to_frame().to_excel(writer, sheet_name="totals_by_severity"))
 
    plt.figure(figsize=(1.2 * max(6, len(sev_order)) + 2,
                        0.38 * max(6, len(mat.index)) + 2))
    ax = sns.heatmap(mat, annot=True, fmt="d", cbar=True)
    ax.set_xlabel("Severity")
    ax.set_ylabel("School")
    ax.set_title("Patients per School by Severity")
    heatmap_path = out_dir / heatmap_png
    plt.tight_layout()
    plt.savefig(heatmap_path, dpi=160, bbox_inches="tight")
    plt.close()

    return mat, heatmap_path

def main():
 
    DATA_XLSX = Path("IEOR4004_HW3_MobileCareData.xlsx")  
    OUT_DIR = Path("Outcome")                                       
    OUT_DIR.mkdir(parents=True, exist_ok=True)
 

    xls, sheet_names = load_workbook(DATA_XLSX)

    # 1) 学校坐标
    schools = parse_schools(DATA_XLSX)

    # 2) 患者 + 严重度
    patients = parse_patients_and_severity(DATA_XLSX)

    # 3) 学校层汇总
    school_summary = build_school_summary(schools, patients)

    # 4) 历史容量估计（每车每日均值 & 月容量）
    capacity = estimate_capacity_from_appointments(DATA_XLSX, sheet_names)

    # 5) 导出 CSV
    schools.to_csv(OUT_DIR / "Q1_schools_clean.csv", index=False)
    patients.to_csv(OUT_DIR / "Q1_patients_with_severity.csv", index=False)
    school_summary.to_csv(OUT_DIR / "Q1_school_summary.csv", index=False)
    capacity.to_csv(OUT_DIR / "Q1_capacity_estimate.csv", index=False)

    # 5.1) 学校 × 严重度矩阵
    mat, heatmap_path = export_school_severity_matrix(patients, OUT_DIR)
    print(f"\nSchool×Severity matrix saved to: {OUT_DIR / 'Q1_school_severity_matrix.xlsx'}")
    print(f"Heatmap saved to: {heatmap_path}")

    print("\n=== EDA Summary ===")
    print(f"Schools: {schools.shape} → saved to {OUT_DIR / 'Q1_schools_clean.csv'}")
    print(f"Patients (long): {patients.shape} → saved to {OUT_DIR / 'Q1_patients_with_severity.csv'}")
    print(f"School-level summary: {school_summary.shape} → {OUT_DIR / 'Q1_school_summary.csv'}")
    print("\nHistorical capacity estimate per van:")
    if len(capacity) > 0:
        print(capacity.to_string(index=False))
        print(f"Saved to {OUT_DIR / 'Q1_capacity_estimate.csv'}")
    else:
        print("No appointment sheets found (Case_SCH#k_APPS).")

if __name__ == "__main__":
    main()


School×Severity matrix saved to: Outcome/Q1_school_severity_matrix.xlsx
Heatmap saved to: Outcome/Q1_heatmap_school_severity.png

=== EDA Summary ===
Schools: (20, 3) → saved to Outcome/Q1_schools_clean.csv
Patients (long): (1600, 5) → saved to Outcome/Q1_patients_with_severity.csv
School-level summary: (20, 8) → Outcome/Q1_school_summary.csv

Historical capacity estimate per van:
   van  daily_avg  monthly_capacity_est
Van #0       16.0                   320
Van #1       16.0                   320
Saved to Outcome/Q1_capacity_estimate.csv


/var/folders/6d/nn3gll_n7f13x_pt18_lrtfc0000gn/T/ipykernel_4228/29267737.py:142: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  patients.assign(


In [3]:
# ===========================
# Q1 Visualization (Append)
# ===========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

OUT_DIR = Path("Outcome") 
schools_csv = OUT_DIR / "Q1_schools_clean.csv"
patients_csv = OUT_DIR / "Q1_patients_with_severity.csv"
school_summary_csv = OUT_DIR / "Q1_school_summary.csv"
capacity_csv = OUT_DIR / "Q1_capacity_estimate.csv"
 
if "schools" not in globals():
    schools = pd.read_csv(schools_csv)
if "patients" not in globals():
    patients = pd.read_csv(patients_csv)
if "school_summary" not in globals():
    school_summary = pd.read_csv(school_summary_csv)
if "capacity" not in globals():
    capacity = pd.read_csv(capacity_csv)

fig1 = plt.figure(figsize=(10, 4))
plt.bar(school_summary["school"], school_summary["num_patients"])
plt.xlabel("School ID")
plt.ylabel("Number of Patients")
plt.title("Patients per School")
plt.tight_layout()
fig1_path = OUT_DIR / "Q1_fig_patients_per_school.png"
plt.savefig(fig1_path, dpi=160, bbox_inches="tight")
plt.close(fig1)
 
sev_cols_all = ["Intermittent", "MildIntermittent", "MildPersistent", "ModeratePersistent", "SeverePersistent"]
sev_cols = [c for c in sev_cols_all if c in school_summary.columns]

fig2 = plt.figure(figsize=(12, 5))
bottom = np.zeros(len(school_summary))
for col in sev_cols:
    vals = school_summary[col].values
    plt.bar(school_summary["school"], vals, bottom=bottom, label=col)
    bottom = bottom + vals
plt.xlabel("School ID")
plt.ylabel("Count")
plt.title("Severity Composition per School (Counts)")
plt.legend(loc="upper right", ncol=2, fontsize=8)
plt.tight_layout()
fig2_path = OUT_DIR / "Q1_fig_severity_stack.png"
plt.savefig(fig2_path, dpi=160, bbox_inches="tight")
plt.close(fig2)
 
fig3 = plt.figure(figsize=(5, 5))
plt.scatter(schools["X"], schools["Y"])
for _, r in schools.iterrows():
    plt.text(r["X"], r["Y"], str(int(r["school_id"])), fontsize=8)
plt.xlabel("X")
plt.ylabel("Y")
plt.title("School Locations (Grid Coordinates)")
plt.tight_layout()
fig3_path = OUT_DIR / "Q1_fig_school_locations.png"
plt.savefig(fig3_path, dpi=160, bbox_inches="tight")
plt.close(fig3)
 
fig4 = plt.figure(figsize=(5, 4))
plt.bar(capacity["van"], capacity["daily_avg"])
plt.xlabel("Van")
plt.ylabel("Daily Avg Appointments")
plt.title("Historical Daily Capacity per Van")
plt.tight_layout()
fig4_path = OUT_DIR / "Q1_fig_capacity_per_van.png"
plt.savefig(fig4_path, dpi=160, bbox_inches="tight")
plt.close(fig4)
 
if "sev_weight" in patients.columns:
    wsum = patients.groupby("school")["sev_weight"].sum().reset_index(name="weighted_sum")
    fig5 = plt.figure(figsize=(10, 4))
    plt.bar(wsum["school"], wsum["weighted_sum"])
    plt.xlabel("School ID")
    plt.ylabel("Weighted Severity Sum")
    plt.title("Weighted Severity Sum per School")
    plt.tight_layout()
    fig5_path = OUT_DIR / "Q1_fig_weighted_severity_per_school.png"
    plt.savefig(fig5_path, dpi=160, bbox_inches="tight")
    plt.close(fig5)
else:
    fig5_path = None

print("[Saved Figures]")
print(f" - {fig1_path}")
print(f" - {fig2_path}")
print(f" - {fig3_path}")
print(f" - {fig4_path}")
if fig5_path:
    print(f" - {fig5_path}")

[Saved Figures]
 - Outcome/Q1_fig_patients_per_school.png
 - Outcome/Q1_fig_severity_stack.png
 - Outcome/Q1_fig_school_locations.png
 - Outcome/Q1_fig_capacity_per_van.png
 - Outcome/Q1_fig_weighted_severity_per_school.png


In [4]:
# q1_optimal_schedule.py
# Solve to optimality (Gurobi) and export a clear, interpretable monthly schedule.
# Objective: maximize total weighted patients served under per-van monthly capacity.

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from gurobipy import Model, GRB, quicksum

# -------------------- Config --------------------
DATA_DIR = Path("Outcome")  # <<<< change to your folder if needed
OUT_DIR  = Path("Outcome")
OUT_DIR.mkdir(parents=True, exist_ok=True)

SCHOOLS_CSV   = DATA_DIR / "Q1_schools_clean.csv"          # school_id, X, Y
PATIENTS_CSV  = DATA_DIR / "Q1_patients_with_severity.csv" # pid, PATIENT, school, SEVERITY, sev_weight
CAPACITY_CSV  = DATA_DIR / "Q1_capacity_estimate.csv"      # van, daily_avg, monthly_capacity_est

VANS = ["Van #0", "Van #1"]  # fixed two vans

# -------------------- Load data --------------------
schools  = pd.read_csv(SCHOOLS_CSV)   # school_id, X, Y
patients = pd.read_csv(PATIENTS_CSV)  # pid, PATIENT, school, SEVERITY, sev_weight
capacity = pd.read_csv(CAPACITY_CSV)  # van, daily_avg, monthly_capacity_est

# sets & maps
S = sorted(schools["school_id"].astype(int).tolist())
P_all = patients["pid"].astype(int).tolist()

s_of_p = patients.set_index("pid")["school"].astype(int).to_dict()
w      = patients.set_index("pid")["sev_weight"].astype(int).to_dict()
severity_map = patients.set_index("pid")["SEVERITY"].to_dict()
name_map     = patients.set_index("pid")["PATIENT"].to_dict()

# van capacity (monthly)
C = {row["van"]: int(row["monthly_capacity_est"]) for _, row in capacity.iterrows()}
for v in VANS:
    if v not in C:  # fallback if missing
        C[v] = int(capacity["monthly_capacity_est"].mean())

# -------------------- Build MILP --------------------
m = Model("MCF_MaxWeightedCoverage")
m.Params.OutputFlag = 1

# decision variables
x = {(s,v): m.addVar(vtype=GRB.BINARY, name=f"x[{s},{v}]") for s in S for v in VANS}  # school->van
y = {p:      m.addVar(vtype=GRB.BINARY, name=f"y[{p}]")     for p in P_all}           # patient served?
z = {(p,v):  m.addVar(vtype=GRB.BINARY, name=f"z[{p},{v}]") for p in P_all for v in VANS}  # patient->van

# objective
m.setObjective(quicksum(w[p]*y[p] for p in P_all), GRB.MAXIMIZE)

# constraints
# (1) each school assigned to exactly one van
for s in S:
    m.addConstr(quicksum(x[(s,v)] for v in VANS) == 1, name=f"one_van_per_school[{s}]")

# (2) y[p] = sum_v z[p,v] <= 1
for p in P_all:
    m.addConstr(y[p] == quicksum(z[(p,v)] for v in VANS), name=f"y_def[{p}]")
    m.addConstr(y[p] <= 1, name=f"y_le_one[{p}]")

# (3) link: patient can only go to van that serves their school
for p in P_all:
    s = s_of_p[p]
    for v in VANS:
        m.addConstr(z[(p,v)] <= x[(s,v)], name=f"link[{p},{v}]")

# (4) van monthly capacity
for v in VANS:
    m.addConstr(quicksum(z[(p,v)] for p in P_all) <= C[v], name=f"capacity[{v}]")

m.optimize()

if m.Status != GRB.OPTIMAL:
    raise RuntimeError(f"Solve status != OPTIMAL (status={m.Status})")

print(f"\nOptimal weighted coverage = {m.ObjVal:.0f}")
print("Van monthly capacities:", C)

# -------------------- Extract solution --------------------
assign = {(s,v): int(round(x[(s,v)].X)) for s in S for v in VANS}
school_to_van = {s: (VANS[0] if assign[(s,VANS[0])]==1 else VANS[1]) for s in S}

rows_pat = []
for p in P_all:
    if y[p].X > 0.5:
        v_star = [v for v in VANS if z[(p,v)].X > 0.5]
        v_use  = v_star[0] if v_star else None
        rows_pat.append({
            "pid": p,
            "PATIENT": name_map[p],
            "school": s_of_p[p],
            "van": v_use,
            "SEVERITY": severity_map.get(p, ""),
            "sev_weight": int(w[p]),
        })

selected_patients = pd.DataFrame(rows_pat).sort_values(["van","school","sev_weight","pid"], ascending=[True, True, False, True])

rows_s = []
for s in S:
    v = school_to_van[s]
    rows_s.append({"school": s, "van": v})
schools_assignment = pd.DataFrame(rows_s).sort_values("school")

# per-school summary
summary = (
    selected_patients
    .groupby(["van","school"])["pid"]
    .count()
    .rename("assigned_patients")
    .reset_index()
)
wsum = (
    selected_patients
    .groupby(["van","school"])["sev_weight"]
    .sum()
    .rename("weighted_sum")
    .reset_index()
)
schedule_summary_by_school = summary.merge(wsum, on=["van","school"], how="outer").fillna(0).sort_values(["van","school"])

# -------------------- Export CSV --------------------
schools_assignment_path = OUT_DIR / "schools_assignment.csv"
selected_patients_path  = OUT_DIR / "selected_patients.csv"
summary_path            = OUT_DIR / "schedule_summary_by_school.csv"

schools_assignment.to_csv(schools_assignment_path, index=False)
selected_patients.to_csv(selected_patients_path, index=False)
schedule_summary_by_school.to_csv(summary_path, index=False)


# ===== 追加：已安排（scheduled）病人按严重度的汇总 =====
# 统一缺失标签
selected_patients["SEVERITY"] = selected_patients["SEVERITY"].fillna("Unknown")

# 1) 全局：按严重度统计已安排的病人数
sev_counts_overall = (
    selected_patients
    .groupby("SEVERITY")["pid"]
    .count()
    .rename("scheduled_count")
    .reset_index()
    .sort_values("scheduled_count", ascending=False)
)
sev_counts_overall_path = OUT_DIR / "scheduled_severity_counts_overall.csv"
sev_counts_overall.to_csv(sev_counts_overall_path, index=False)

# 2) 分车（van）+严重度：每辆车各严重度的已安排人数（透视表）
sev_counts_by_van = (
    selected_patients
    .groupby(["van", "SEVERITY"])["pid"]
    .count()
    .rename("scheduled_count")
    .reset_index()
    .pivot(index="SEVERITY", columns="van", values="scheduled_count")
    .fillna(0)
    .astype(int)
    .sort_index()
)
sev_counts_by_van_path = OUT_DIR / "scheduled_severity_counts_by_van.csv"
sev_counts_by_van.to_csv(sev_counts_by_van_path)

# 3) （可选）分车×分校×严重度：更细的明细表
sev_counts_by_van_school = (
    selected_patients
    .groupby(["van", "school", "SEVERITY"])["pid"]
    .count()
    .rename("scheduled_count")
    .reset_index()
    .sort_values(["van", "school", "SEVERITY"])
)
sev_counts_by_van_school_path = OUT_DIR / "scheduled_severity_counts_by_van_school.csv"
sev_counts_by_van_school.to_csv(sev_counts_by_van_school_path, index=False)

# 控制台打印一个简洁汇总
print("\n[Scheduled Patients by Severity]")
print(sev_counts_overall.to_string(index=False))

# （可选）算一个平均严重度，便于报告
try:
    avg_sev_among_scheduled = (
        selected_patients["sev_weight"].sum() / len(selected_patients)
        if len(selected_patients) > 0 else 0.0
    )
    print(f"Avg severity weight (scheduled): {avg_sev_among_scheduled:.3f}")
except Exception:
    pass

print("Saved:", sev_counts_overall_path)
print("Saved:", sev_counts_by_van_path)
print("Saved:", sev_counts_by_van_school_path)

print("Saved:", schools_assignment_path)
print("Saved:", selected_patients_path)
print("Saved:", summary_path)

# -------------------- Visualizations --------------------
# 1) School assignment map (scatter), label by school id, annotate van
fig1 = plt.figure(figsize=(6,6))
sch_plot = schools.merge(schools_assignment, left_on="school_id", right_on="school", how="left")
plt.scatter(sch_plot["X"], sch_plot["Y"])
for _, r in sch_plot.iterrows():
    label = f"{int(r['school_id'])}\n({r['van']})"
    plt.text(r["X"], r["Y"], label, fontsize=8)
plt.xlabel("X"); plt.ylabel("Y"); plt.title("School-to-Van Assignment (Map)")
plt.tight_layout()
fig1_path = OUT_DIR / "fig_assignment_map.png"
plt.savefig(fig1_path, dpi=160, bbox_inches="tight")
plt.close(fig1)

# 2) Assigned patients per school by van (side-by-side bars)
wide_counts = schedule_summary_by_school.pivot(index="school", columns="van", values="assigned_patients").fillna(0).reindex(S)
fig2 = plt.figure(figsize=(10,4))
ind = np.arange(len(wide_counts.index))
width = 0.35
vals0 = wide_counts[VANS[0]].values if VANS[0] in wide_counts.columns else np.zeros(len(ind))
vals1 = wide_counts[VANS[1]].values if VANS[1] in wide_counts.columns else np.zeros(len(ind))
plt.bar(ind - width/2, vals0, width, label=VANS[0])
plt.bar(ind + width/2, vals1, width, label=VANS[1])
plt.xticks(ind, wide_counts.index, rotation=0)
plt.xlabel("School"); plt.ylabel("Assigned Patients"); plt.title("Assigned Patients per School (by Van)")
plt.legend()
plt.tight_layout()
fig2_path = OUT_DIR / "fig_assigned_counts_by_school.png"
plt.savefig(fig2_path, dpi=160, bbox_inches="tight")
plt.close(fig2)

# 3) Weighted sum per school (by van)
wide_w = schedule_summary_by_school.pivot(index="school", columns="van", values="weighted_sum").fillna(0).reindex(S)
fig3 = plt.figure(figsize=(10,4))
ind = np.arange(len(wide_w.index))
width = 0.35
vals0 = wide_w[VANS[0]].values if VANS[0] in wide_w.columns else np.zeros(len(ind))
vals1 = wide_w[VANS[1]].values if VANS[1] in wide_w.columns else np.zeros(len(ind))
plt.bar(ind - width/2, vals0, width, label=VANS[0])
plt.bar(ind + width/2, vals1, width, label=VANS[1])
plt.xticks(ind, wide_w.index, rotation=0)
plt.xlabel("School"); plt.ylabel("Weighted Sum"); plt.title("Weighted Coverage per School (by Van)")
plt.legend()
plt.tight_layout()
fig3_path = OUT_DIR / "fig_weighted_sum_by_school.png"
plt.savefig(fig3_path, dpi=160, bbox_inches="tight")
plt.close(fig3)

# 4) Top-K schools for each van (by assigned count)
K = 5
tops = []
for v in VANS:
    dfv = schedule_summary_by_school[schedule_summary_by_school["van"]==v].sort_values("assigned_patients", ascending=False).head(K)
    dfv = dfv.assign(rank=np.arange(1, len(dfv)+1))
    tops.append(dfv)
topk = pd.concat(tops, ignore_index=True)
topk_path = OUT_DIR / "topk_schools_by_van.csv"
topk.to_csv(topk_path, index=False)

print("Saved figures:")
print(fig1_path)
print(fig2_path)
print(fig3_path)
print("Saved:", topk_path)

print("\nDONE. Provide these in your Results section:")
print("- Optimal objective (weighted coverage):", int(round(m.ObjVal))) 

Set parameter Username
Set parameter LicenseID to value 2722020
Academic license - for non-commercial use only - expires 2026-10-14
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A354)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 6422 rows, 4840 columns and 16040 nonzeros
Model fingerprint: 0x579d41c6
Variable types: 0 continuous, 4840 integer (4840 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 1620 rows and 1620 columns
Presolve time: 0.02s
Presolved: 4802 rows, 3220 columns, 12800 nonzeros
Variable types: 0 continuous, 3220 integer (3220 binary)
Found heuristic solution: objective 1417.0000000

Root relaxation: objective 1.923000e+03, 2741 iterations, 0.07 s

In [5]:
# routing_tsp_per_van_combined.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from pathlib import Path
from math import sqrt
 
DATA_DIR = Path("Outcome")   
OUT_DIR  = Path("Outcome")
OUT_DIR.mkdir(parents=True, exist_ok=True)

SCHOOLS_CSV  = DATA_DIR / "Q1_schools_clean.csv"   
ASSIGN_CSV   = DATA_DIR / "schools_assignment.csv"    
DIST_UNIT    = "euclidean" 
UNIT_SCALE   = 0.1           
MAKE_CYCLE   = True    
FIXED_START  = None         # 
VANS_ORDER   = ["Van #0", "Van #1"]   

# ---------- Load ----------
schools = pd.read_csv(SCHOOLS_CSV)   
assign  = pd.read_csv(ASSIGN_CSV)       
df = assign.merge(schools, left_on="school", right_on="school_id", how="left")
df = df[["school", "van", "X", "Y"]].sort_values(["van","school"]).reset_index(drop=True)

# ---------- Distance helpers ----------
def dist(a, b, metric="euclidean", scale=1.0):
    if metric == "euclidean":
        d = sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)
    elif metric == "manhattan":
        d = abs(a[0]-b[0]) + abs(a[1]-b[1])
    else:
        raise ValueError("Unknown metric")
    return d * scale

def build_complete_graph(nodes_xy, metric="euclidean", scale=1.0):
    G = nx.Graph()
    for n, (x, y) in nodes_xy.items():
        G.add_node(n, pos=(x, y))
    ids = list(nodes_xy.keys())
    for i in range(len(ids)):
        for j in range(i+1, len(ids)):
            u, v = ids[i], ids[j]
            w = dist(nodes_xy[u], nodes_xy[v], metric, scale)
            G.add_edge(u, v, weight=w)
    return G

def route_length(route, nodes_xy, metric="euclidean", scale=1.0, cycle=True):
    total = 0.0
    for i in range(len(route)-1):
        total += dist(nodes_xy[route[i]], nodes_xy[route[i+1]], metric, scale)
    if cycle and len(route) > 1:
        total += dist(nodes_xy[route[-1]], nodes_xy[route[0]], metric, scale)
    return total

def solve_tsp_sequence(nodes_xy, metric="euclidean", scale=1.0, cycle=True, fixed_start=None):
    if len(nodes_xy) <= 1:
        return list(nodes_xy.keys())
    G = build_complete_graph(nodes_xy, metric, scale)
    tour = nx.approximation.traveling_salesman_problem(
        G, weight="weight", cycle=cycle, method=nx.approximation.christofides
    )
    if cycle and len(tour) > 1 and tour[0] == tour[-1]:
        tour = tour[:-1]
    # rotate to fixed start if provided; else use left-most as start for consistent labels
    if fixed_start is None or fixed_start not in nodes_xy:
        fixed_start = min(nodes_xy, key=lambda s: nodes_xy[s][0])
    if fixed_start in tour:
        idx = tour.index(fixed_start)
        tour = tour[idx:] + tour[:idx]
    return tour

# ---------- Solve per van & export ----------
all_routes = {}   # van -> {"path": [...], "points": {...}, "length": float}
color_map  = {"Van #0": "tab:blue", "Van #1": "tab:orange"}

for van, grp in df.groupby("van"):
    points = {int(r.school): (float(r.X), float(r.Y)) for _, r in grp.iterrows()}
    if len(points) == 0:
        continue
    path = solve_tsp_sequence(points, metric=DIST_UNIT, scale=UNIT_SCALE, cycle=MAKE_CYCLE, fixed_start=FIXED_START)
    L = route_length(path, points, metric=DIST_UNIT, scale=UNIT_SCALE, cycle=MAKE_CYCLE)
    all_routes[van] = {"path": path, "points": points, "length": L}

    # 导出 CSV
    rows = []
    cum = 0.0
    for i in range(len(path)):
        s = path[i]
        step = 0.0 if i == 0 else dist(points[path[i-1]], points[s], DIST_UNIT, UNIT_SCALE)
        cum += step
        rows.append({"order": i+1, "school": s, "x": points[s][0], "y": points[s][1], "step_dist": step, "cum_dist": cum})
    if MAKE_CYCLE and len(path) > 1:
        back = dist(points[path[-1]], points[path[0]], DIST_UNIT, UNIT_SCALE)
        rows.append({"order": len(path)+1, "school": path[0], "x": points[path[0]][0], "y": points[path[0]][1],
                     "step_dist": back, "cum_dist": cum + back})
    pd.DataFrame(rows).to_csv(OUT_DIR / (van.replace(" ", "").lower() + "_route.csv"), index=False)

    # 单独图（可保留）
    xs = [points[s][0] for s in path] + ([points[path[0]][0]] if MAKE_CYCLE and len(path)>1 else [])
    ys = [points[s][1] for s in path] + ([points[path[0]][1]] if MAKE_CYCLE and len(path)>1 else [])
    fig = plt.figure(figsize=(6,6))
    plt.scatter([p[0] for p in points.values()], [p[1] for p in points.values()], color=color_map.get(van, None))
    plt.plot(xs, ys, linewidth=2, color=color_map.get(van, None))
    for i, s in enumerate(path, start=1):
        plt.text(points[s][0], points[s][1], f"{s}\n#{i}", fontsize=8, color=color_map.get(van, None))
    plt.xlabel("X"); plt.ylabel("Y"); plt.title(f"{van} - Shortest Route (approx. TSP)")
    plt.tight_layout()
    plt.savefig(OUT_DIR / (van.replace(" ", "").lower() + "_route.png"), dpi=160, bbox_inches="tight")
    plt.close(fig)

# ---------- Combined figure ----------
fig = plt.figure(figsize=(7,7))

# 
all_X = [float(r.X) for _, r in df.iterrows()]
all_Y = [float(r.Y) for _, r in df.iterrows()]
plt.scatter(all_X, all_Y, s=18, alpha=0.25, label="All schools")

for van in VANS_ORDER:
    if van not in all_routes: 
        continue
    path   = all_routes[van]["path"]
    points = all_routes[van]["points"]
    col    = color_map.get(van, None)

    xs = [points[s][0] for s in path]
    ys = [points[s][1] for s in path]
    if MAKE_CYCLE and len(path) > 1:
        xs = xs + [points[path[0]][0]]
        ys = ys + [points[path[0]][1]]

    plt.plot(xs, ys, linewidth=2, label=f"{van} (len={all_routes[van]['length']:.1f})", color=col)
    plt.scatter([points[s][0] for s in path], [points[s][1] for s in path], s=30, color=col)
    for i, s in enumerate(path, start=1):
        plt.text(points[s][0], points[s][1], f"{s}\n#{i}", fontsize=8, color=col)

plt.xlabel("X"); plt.ylabel("Y")
plt.title("Both Vans - Shortest Routes (Combined)")
plt.legend()
plt.tight_layout()

combined_png = OUT_DIR / "fig_routes_combined.png"
plt.savefig(combined_png, dpi=160, bbox_inches="tight")
plt.close(fig)

print("Saved combined figure:", combined_png)

Saved combined figure: Outcome/fig_routes_combined.png
